In [1]:
# ! ansible-inventory --list

In [2]:
! ansible-playbook playbooks/server/create.yml

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [Create servers] **********************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [ansible.builtin.file] ****************************************************
changed: [localhost]

TASK [Generate an OpenSSH keypair with the default values (4096 bits, rsa)] ****
changed: [localhost]

TASK [Remove ssh_key] **********************************************************
ok: [localhost]

TASK [Add new ssh_key] *********************************************************
changed: [localhost]

TASK [Create servers] **********************************************************
changed: [localhost] => (item=controller-0)
changed: [localhost] => (item=controller-1)
changed: [localhost] => (item=controller-2)
changed: [localhost] => (item=worker-0)
changed: [localhost] => (item=worke

In [3]:
! ansible-playbook playbooks/server/ssh.yml


PLAY [Create ssh config] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [ssh : set_fact] **********************************************************
ok: [localhost]

TASK [ssh : Write ssh config file] *********************************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



## Provisioning a CA and Generating TLS Certificates
[Provisioning a CA and Generating TLS Certificates](https://github.com/kelseyhightower/kubernetes-the-hard-way/blob/master/docs/04-certificate-authority.md#provisioning-a-ca-and-generating-tls-certificates)

In [4]:
! ansible-playbook playbooks/provision.yml


PLAY [Provision k8s cluster] ***************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-0]

TASK [certs : Install cfssl] ***************************************************
changed: [controller-0]

TASK [certs : Install cfssljson] ***********************************************
changed: [controller-0]

TASK [certs : Create certs dir] ************************************************
changed: [controller-0]

TASK [certs : Copy cfssl template files] ***************************************
changed: [controller-0] => (item=ca-config.json)
changed: [controller-0] => (item=ca-csr.json)
changed: [controller-0] => (item=admin-csr.json)
changed: [controller-0] => (item=kube-controller-manager-csr.json)
changed: [controller-0] => (item=kube-scheduler-csr.json)
changed: [controller-0] => (item=kube-proxy-csr.json)
changed: [controller-0] => (item=kubernetes-csr.json)
changed: [controller-0] => (item=service

In [5]:
! ansible-playbook playbooks/distribute.yml


PLAY [Fetch files] *************************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-0]

TASK [Download certs and kubeconfigs to local machine] *************************
changed: [controller-0]

PLAY [Distribute certs to worker] **********************************************

TASK [Gathering Facts] *********************************************************
ok: [worker-0]
ok: [worker-2]
ok: [worker-1]

TASK [Copy worker certs and kubeconfigs] ***************************************
changed: [worker-0] => (item=-key.pem)
changed: [worker-2] => (item=-key.pem)
changed: [worker-1] => (item=-key.pem)
changed: [worker-2] => (item=.pem)
changed: [worker-0] => (item=.pem)
changed: [worker-1] => (item=.pem)
changed: [worker-0] => (item=.kubeconfig)
changed: [worker-2] => (item=.kubeconfig)
changed: [worker-1] => (item=.kubeconfig)

TASK [Copy ca.pem] ****************************************************

In [6]:
! ansible-playbook playbooks/bootstrap.yml --tags=etcd


PLAY [Deploy to controllers] ***************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-2]
ok: [controller-0]
ok: [controller-1]

TASK [etcd : Fetch etcd binaries] **********************************************
changed: [controller-0]
changed: [controller-1]
changed: [controller-2]

TASK [etcd : Install etcd binaries] ********************************************
changed: [controller-1] => (item=etcd)
changed: [controller-2] => (item=etcd)
changed: [controller-0] => (item=etcd)
changed: [controller-1] => (item=etcdctl)
changed: [controller-2] => (item=etcdctl)
changed: [controller-0] => (item=etcdctl)

TASK [etcd : ansible.builtin.shell] ********************************************
changed: [controller-2]
changed: [controller-1]
changed: [controller-0]

TASK [etcd : set_fact] *********************************************************
ok: [controller-0]
ok: [controller-1]
ok: [controller-2]

TASK [

In [7]:
# Verify that the etcd cluster is running
! ansible controller-0 -a 'etcdctl member list --cert=/etc/etcd/kubernetes.pem --key=/etc/etcd/kubernetes-key.pem --cacert=/etc/etcd/ca.pem'

controller-0 | CHANGED | rc=0 >>
274046df03c6ae53, started, controller-0, https://xxx.xxx.67.57:2380, https://xxx.xxx.67.57:2379, false
48f67b44febe0ffd, started, controller-2, https://xxx.xxx.118.255:2380, https://xxx.xxx.118.255:2379, false
714bcb2ef90babee, started, controller-1, https://xxx.xxx.175.63:2380, https://xxx.xxx.175.63:2379, false


In [ ]:
! ansible-playbook playbooks/bootstrap.yml --tags=control-plane

In [9]:
# verification
! ansible controller-0 -a 'kubectl cluster-info --kubeconfig admin.kubeconfig'

controller-0 | CHANGED | rc=0 >>
Kubernetes control plane is running at https://127.0.0.1:6443

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [10]:
! ansible controller-0 -a 'curl --cacert ca.pem https://127.0.0.1:6443/version'

controller-0 | CHANGED | rc=0 >>
{
  "major": "1",
  "minor": "21",
  "gitVersion": "v1.21.0",
  "gitCommit": "cb303e613a121a29364f75cc67d3d580833a7479",
  "gitTreeState": "clean",
  "buildDate": "2021-04-08T16:25:06Z",
  "goVersion": "go1.16.1",
  "compiler": "gc",
  "platform": "linux/amd64"
}  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   263  100   263    0     0   8766      0 --:--:-- --:--:-- --:--:--  9068


In [11]:
! ansible-playbook playbooks/bootstrap.yml --tags=rbac


PLAY [Deploy to controllers] ***************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-1]
ok: [controller-0]
ok: [controller-2]

PLAY [Deploy to controller 0] **************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-0]

TASK [rbac : Create the system:kube-apiserver-to-kubelet ClusterRole with permissions to access the Kubelet API and perform most common tasks associated with managing pods] ***
changed: [controller-0]

TASK [rbac : Bind the system:kube-apiserver-to-kubelet ClusterRole to the kubernetes user] ***
changed: [controller-0]

PLAY [Deploy to workers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [worker-0]
ok: [worker-1]
ok: [worker-2]

PLAY RECAP *****************************************************

In [12]:
# verification of cluster role
! ansible controller-0 -a 'kubectl describe clusterrole system:kube-apiserver-to-kubelet --kubeconfig admin.kubeconfig'

controller-0 | CHANGED | rc=0 >>
Name:         system:kube-apiserver-to-kubelet
Labels:       kubernetes.io/bootstrapping=rbac-defaults
Annotations:  rbac.authorization.kubernetes.io/autoupdate: true
PolicyRule:
  Resources      Non-Resource URLs  Resource Names  Verbs
  ---------      -----------------  --------------  -----
  nodes/log      []                 []              [*]
  nodes/metrics  []                 []              [*]
  nodes/proxy    []                 []              [*]
  nodes/spec     []                 []              [*]
  nodes/stats    []                 []              [*]


In [13]:
! ansible-playbook playbooks/bootstrap.yml --tags=worker-nodes


PLAY [Deploy to controllers] ***************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-1]
ok: [controller-2]
ok: [controller-0]

PLAY [Deploy to controller 0] **************************************************

TASK [Gathering Facts] *********************************************************
ok: [controller-0]

PLAY [Deploy to workers] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [worker-1]
ok: [worker-0]
ok: [worker-2]

TASK [worker-nodes : Install the OS dependencies] ******************************
changed: [worker-0] => (item=socat)
changed: [worker-2] => (item=socat)
changed: [worker-1] => (item=socat)
changed: [worker-0] => (item=conntrack)
changed: [worker-2] => (item=conntrack)
changed: [worker-1] => (item=conntrack)
changed: [worker-0] => (item=ipset)
changed: [worker-2] => (item=ipset)
chang

In [14]:
! ansible-playbook playbooks/objects.yml


PLAY [Install local tools] *****************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [kubectl : Configure kubectl] *********************************************
changed: [localhost]

TASK [dns : Deploy coredns] ****************************************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



## DNS

In [15]:
#! kubectl describe po $(kubectl get po -o jsonpath="{.items[0].metadata.name}" -n kube-system) -n kube-system
! kubectl logs $(kubectl get po -o jsonpath="{.items[0].metadata.name}" -n kube-system) -n kube-system

In [16]:
# 
! kubectl run busybox --image=busybox:1.28 --command -- sleep 3600

pod/busybox created


In [17]:
! kubectl get pods -l run=busybox

NAME      READY   STATUS    RESTARTS   AGE
busybox   0/1     Pending   0          0s


In [20]:
! kubectl exec -ti busybox -- nslookup kube-dns.kube-system.svc.cluster.local

Server:    10.32.0.10
Address 1: 10.32.0.10

nslookup: can't resolve 'kube-dns.kube-system.svc.cluster.local'
command terminated with exit code 1


In [56]:
## @todo: Fix DNS (see above)

## Smoke tests

[Smoke tests](https://github.com/kelseyhightower/kubernetes-the-hard-way/blob/master/docs/13-smoke-test.md#smoke-test)

### Data Encryption

In [59]:
# Create a generic secret
! kubectl create secret generic kubernetes-the-hard-way \
  --from-literal="mykey=mydata"

secret/kubernetes-the-hard-way created


In [62]:
# Print a hexdump of the kubernetes-the-hard-way secret stored in etcd
! ssh -q controller-0 \
  "sudo ETCDCTL_API=3 etcdctl get \
  --endpoints=https://127.0.0.1:2379 \
  --cacert=/etc/etcd/ca.pem \
  --cert=/etc/etcd/kubernetes.pem \
  --key=/etc/etcd/kubernetes-key.pem\
  /registry/secrets/default/kubernetes-the-hard-way | hexdump -C"

00000000  2f 72 65 67 69 73 74 72  79 2f 73 65 63 72 65 74  |/registry/secret|
00000010  73 2f 64 65 66 61 75 6c  74 2f 6b 75 62 65 72 6e  |s/default/kubern|
00000020  65 74 65 73 2d 74 68 65  2d 68 61 72 64 2d 77 61  |etes-the-hard-wa|
00000030  79 0a 6b 38 73 3a 65 6e  63 3a 61 65 73 63 62 63  |y.k8s:enc:aescbc|
00000040  3a 76 31 3a 6b 65 79 31  3a 0c eb 75 2e 4e 2a 0f  |:v1:key1:..u.N*.|
00000050  57 8b 2b d4 bb 39 bc 80  4a 53 4d 91 88 02 02 26  |W.+..9..JSM....&|
00000060  45 7f 83 1a 01 be 23 1a  e4 a5 37 ff 4c d5 79 05  |E.....#...7.L.y.|
00000070  86 aa 00 ae 0a 50 40 7f  4b 61 58 c6 28 c8 08 87  |.....P@.KaX.(...|
00000080  25 dc f6 72 11 23 cb 70  df 37 5a dd e6 90 8f ea  |%..r.#.p.7Z.....|
00000090  48 74 f2 a7 3d 7e 63 09  4e 4e e3 7e dc fc 57 26  |Ht..=~c.NN.~..W&|
000000a0  5f 4b 93 40 56 7c 3d b9  62 68 0a cc c6 44 50 84  |_K.@V|=.bh...DP.|
000000b0  00 54 a3 cb da 91 68 59  14 03 ee 60 88 aa 92 8a  |.T....hY...`....|
000000c0  4a 38 7a c7 e1 1e 4b 57  0a fb 58 ce e6 d9

The etcd key should be prefixed with k8s:enc:aescbc:v1:key1, which indicates the aescbc provider was used to encrypt the data with the key1 encryption key.

### Deployments

In [64]:
! kubectl create deployment nginx --image=nginx

deployment.apps/nginx created


In [65]:
! kubectl get pods -l app=nginx

NAME                     READY   STATUS    RESTARTS   AGE
nginx-6799fc88d8-tj7xq   1/1     Running   0          8s


In [66]:
# Port forwarding can't be tested within this Jupyter notebook because background processes are not allowed
# But I could succesfully test it outside of Jupyter
# kubectl port-forward $POD_NAME 8080:80

In [75]:
# Checking the logs
! kubectl logs nginx-6799fc88d8-tj7xq | tail -1

127.0.0.1 - - [23/Jun/2022:08:39:04 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.79.1" "-"


In [76]:
# Running a command in the pod
! kubectl exec -ti nginx-6799fc88d8-tj7xq -- nginx -v

nginx version: nginx/1.23.0


### Services

In [77]:
! kubectl expose deployment nginx --port 80 --type NodePort

service/nginx exposed


In [78]:
! kubectl get svc nginx --output=jsonpath='{range .spec.ports[0]}{.nodePort}'

32744

In [98]:
# Using curl from controller-0 to acces the public IP of worker-1 where the nginx pod runs
! ssh -q controller-0 "curl -Is http://worker-1:32744"

HTTP/1.1 200 OK
Server: nginx/1.23.0
Date: Thu, 23 Jun 2022 08:53:54 GMT
Content-Type: text/html
Content-Length: 615
Last-Modified: Tue, 21 Jun 2022 14:25:37 GMT
Connection: keep-alive
ETag: "62b1d4e1-267"
Accept-Ranges: bytes

